In [36]:
import sys
import os
# !{sys.executable} -m pip install -U langchain --no-warn-script-location > /dev/null
# !pip install lancedb
# !pip install llama-parse
# !pip install llama-index
# !pip install predictionguard
# !pip install sentence-transformers
# !pip install momento
# !pip install flask
!pip install flask-cors
# !pip install flask-ngrok
# !pip install pyngrok

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage
from langchain.llms import PredictionGuard
from langchain.memory import ChatMessageHistory
import os

os.environ['PREDICTIONGUARD_TOKEN'] = "q1VuOjnffJ3NO2oFN8Q9m8vghYc84ld13jaqdF7E"
os.environ['MOMENTO_API_KEY'] = "eyJlbmRwb2ludCI6ImNlbGwtNC11cy13ZXN0LTItMS5wcm9kLmEubW9tZW50b2hxLmNvbSIsImFwaV9rZXkiOiJleUpoYkdjaU9pSklVekkxTmlKOS5leUp6ZFdJaU9pSnRZVzVwYTJGdWRHRnpZVzVxWVhreE9UazVRR2R0WVdsc0xtTnZiU0lzSW5abGNpSTZNU3dpY0NJNklrTkJRVDBpTENKbGVIQWlPakUzTVRBNE1ESTJPVGg5LjVLZm9HX1JGck02dmZqcTRQUU5QV0lVdXVFaHVZb2JBRWhSell4X1V0WGcifQ=="

In [2]:
import nest_asyncio
nest_asyncio.apply()
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key="llx-VqWwj6hJue36ZZ5UhU0hufc1ArT6p3L3vqXI3zgQfH3jpzzs",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True
)

def load_document(path):
  documents = parser.load_data(path)
  text = documents[0].text

  # Clean things up just a bit.
  text = text.split("### I. PERSONAL INFORMATION")[1]
  return text

# os.mkdir(".lancedbq")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd
import lancedb
from lancedb.embeddings import with_embeddings

model = SentenceTransformer("all-MiniLM-L12-v2")
def embed_batch(batch):
    res = [model.encode(sentence) for sentence in batch]
    return res
def embed(sentence):
    return model.encode(sentence)

def data_cleanup_and_formation(data, user_id):
  if(len(data) == 0):
    return pd.DataFrame()
  # Chunk the text into smaller pieces for injection into LLM prompts.
  text_splitter = CharacterTextSplitter(chunk_size=700, chunk_overlap=50, separator="##")
  docs = text_splitter.split_text(data)

  # Let's checkout some of the chunks!
  # for i in range(0, len(docs)):
  #   print("Chunk", str(i+1))
  #   print("----------------------------")
  #   print(docs[i])
  #   print("")

  docs = [x.replace('#', '-') for x in docs]
  docs = [x.replace('&emsp', '') for x in docs]

  metadata = []
  for i in range(len(docs)):
      metadata.append([
          user_id,
          i,
          docs[i]
      ])
  doc_df = pd.DataFrame(metadata, columns=["user_id", "chunk", "text"])

  # Embed the documents
  data = with_embeddings(embed_batch, doc_df)
  print("data: ", type(data))
  return data

##### Lance DB setup
uri = ".lancedbq"
db = lancedb.connect(uri)

import pyarrow as pa
def save_user_data(user_id, text):
  data = data_cleanup_and_formation(text, user_id)
  # Create the DB table and add the records.
  try:
    db.create_table("counsellor", data)
  except Exception as e:
    print("Table counsellor does not exists")
  table = db.open_table("counsellor")
  #print("my table: ", type(table))
  table.add(data=data)
  return data

def retrieve_embeddings(user_id, message):
  table = db.open_table("counsellor")
  results = table.search(embed(message)).limit(2).to_df()
  return results
  results.head()

/home/u533a18c56193e287597492a1c516758/.local/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/u533a18c56193e287597492a1c516758/.local/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# Let's try to match a query to one of our documents.
# Let's try to match a query to one of our documents.
text = load_document("./Pre-Counseling_questionnaire_answers.pdf")
data = save_user_data("user1111", text)
print(data)
message = "What are the more severe issues being faced?"
results = retrieve_embeddings("user1111", message)
results.head()

Started parsing the file under job_id b747f5f5-0050-4171-8837-b19cdf52deb7


Created a chunk of size 1403, which is longer than the specified 700
Created a chunk of size 1667, which is longer than the specified 700
Created a chunk of size 2870, which is longer than the specified 700


data:  <class 'pyarrow.lib.Table'>
Table counsellor does not exists
pyarrow.Table
user_id: string
chunk: int64
text: string
vector: fixed_size_list<item: float>[384]
  child 0, item: float
----
user_id: [["user1111","user1111","user1111","user1111","user1111","user1111"]]
chunk: [[0,1,2,3,4,5]]
text: [["Name: Sanjay ; Birth Date: July 20, 1995

Address: 100 E San Fernando St, San Jose, California

Employer: Unemployed ; Occupation: Student ; Highest Education: Bachelors

Cell Phone: 347-264-6764 ; Work Phone: __________ ; Land Line: __________","- II. FAMILY HISTORY

A) Briefly describe the atmosphere and relationships you experienced during your childhood. Information about closeness to the family, who exerted authority, the roles of religion, the extended family, education, and health will be helpful.

Growing up, my childhood was marked by a complex family dynamic that fluctuated between warmth and tension. My parents, while loving, often disagreed on disciplinary measures, leading 

/tmp/ipykernel_58440/3351683328.py:64: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(embed(message)).limit(2).to_df()


,chunk,text,vector,_distance
0,5,VI. CURRENT PROBLEMS\n\nBelow is a list of fre...,"[-0.012839996, -0.040237296, 0.02038348, 0.060...",1.001306
1,5,VI. CURRENT PROBLEMS\n\nBelow is a list of fre...,"[-0.012839996, -0.040237296, 0.02038348, 0.060...",1.001306


In [5]:
from datetime import timedelta
from langchain.memory import MomentoChatMessageHistory
cache_name = "langchain"
ttl = timedelta(days=1)

In [6]:
import predictionguard as pg
from langchain import PromptTemplate

llm=PredictionGuard(model="Neural-Chat-7B")

# Now let's augment our Q&A prompt with this external knowledge on-the-fly!!!
template_start = """
Current conversation:
{history}
### Instruction:
Assume you are a mental health professional with 10+ years experience. You will get chat based questions from a patient. Based on the question, you must generate responses for the user.
Make absolutely sure all your responses seem like actual responses from a mental health expert.
Do not suggest the patient to seek alternatives or to seek other counsellors or therapists or mental health
The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).
"""


template_summary = """
### Input:
Context: {context}

Question: {question}

### Output:
<reply>
"""

template_chat = """
Current conversation:
{history}
The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).

### Input:
Context: {context}

Question: {question}

### Output:
<reply>
"""

# Create a ConversationChain instance with a ConversationBufferMemory
#chain = ConversationChain(memory=ConversationBufferMemory(), llm=llm)

users_history = {}
users_chain = {} 
 

def init_user_history(user_id):
  instruction = '''Assume you are a mental health professional with 10+ years experience. You will get chat based questions from a patient. Based on the question, you must generate responses for the user.
  Make absolutely sure all your responses seem like actual responses from a mental health expert.
  Do not suggest the patient to seek alternatives or to seek other counsellors or therapists or mental health
  The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).'''
  users_chain[user_id]=ConversationChain(
    llm=llm,
    verbose=True
)
  users_history[user_id] = MomentoChatMessageHistory.from_client_params(
    user_id,
    cache_name,
    ttl,
  )
  if len(users_history[user_id].messages)==0:
      users_history[user_id].add_user_message(instruction)
      users_history[user_id].add_ai_message("")

import math
def getMessageHistory(user_id, context, message):
  len_needed = len(context) + len(message)
  remaining_len = min(3000 - len_needed, len(message))
  return users_history[user_id].messages[-remaining_len:]

def get_user_summary(user_id, user_text):
  instruction = '''Assume you are a mental health professional with 10+ years experience. You will get chat based questions from a patient. Based on the question, you must generate responses for the user.
  Make absolutely sure all your responses seem like actual responses from a mental health expert.
  Do not suggest the patient to seek alternatives or to seek other counsellors or therapists or mental health
  The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).'''
  question = "Based on the text below, summarize the patient's key points in 200 words: \n\n " + user_text
    
  qa_prompt = PromptTemplate(
      input_variables=["context", "question"],
      template=template_summary,
  )

  # Augment the prompt with the context
  prompt = qa_prompt.format(context=instruction, question=question)
  result = users_chain[user_id].predict(input=prompt)
  return result

def rag_answer(user_id, message, template=template_chat):
  table = db.open_table("counsellor")
  qa_prompt = PromptTemplate(
      input_variables=["history", "context", "question"],
      template=template,
  )

  # Search the for relevant context
  results = retrieve_embeddings(user_id, message)
  #results = table.search(embed(message)).limit(2).to_df()
  results.sort_values(by=['_distance'], inplace=True, ascending=True)
  doc_use = results['text'].values[0]

  # Augment the prompt with the context
  prompt = qa_prompt.format(context=doc_use, question=message, history=getMessageHistory(user_id, doc_use, message))
  # Get a response
  # result = pg.Completion.create(
  #     model="Nous-Hermes-Llama2-13B"
  # )
  # chain.add_user_message(message)

  result = users_chain[user_id].predict(input=prompt)
  return result

  # return result['choices'][0]['text']

def save_conversation_to_history(user_id, user_reply, counselor_reply):
  users_history[user_id].add_user_message(user_reply)
  users_history[user_id].add_ai_message(counselor_reply)


In [7]:
# user_text = load_document("./Pre-Counseling_questionnaire_answers.pdf")
# user_summary = get_user_summary("user1111",user_text)
# print("Summary: ", user_summary)

Started parsing the file under job_id 7e5214b2-5d91-4bb7-a2e5-26495c91bb8c


KeyError: 'user1111'

In [9]:
# ## Usage
# user_id="user1111"
# init_user_history("user1111")
# response = rag_answer(user_id, "", template=template_start)
# save_conversation_to_history(user_id, "", response)
# print('')
# print("RESPONSE:", response)
# # response = rag_answer("What is the age of Sanjay?")
# # print('')
# # print("RESPONSE:", response)

# response = rag_answer(user_id, "I have depression.")
# save_conversation_to_history(user_id, "I have depression.", response)
# print('')
# print("RESPONSE:", response)

# response = rag_answer(user_id, "I like red colours and hate blue colours.")
# save_conversation_to_history(user_id, "I like red colours and hate blue colours.", response)
# print('')
# print("RESPONSE:", response)
# print("HISTORY:",users_history[user_id])

/tmp/ipykernel_46178/3351683328.py:64: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(embed(message)).limit(2).to_df()




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
Current conversation:
[HumanMessage(content='Assume you are a mental health professional with 10+ years experience. You will get chat based questions from a patient. Based on the question, you must generate responses for the user.\n  Make absolutely sure all your responses seem like actual responses from a mental health expert.\n  Do not suggest the patient to seek alternatives or to seek other counsellors or therapists or mental health\n  The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).'), AIMessage(content=''), HumanMessage(content=''), AIMessage(content="<reply> It is important for m

/tmp/ipykernel_46178/3351683328.py:64: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(embed(message)).limit(2).to_df()



> Finished chain.

RESPONSE: Depression can indeed affect various aspects of your life, but it is important for us to evaluate your symptoms and explore specific behaviors and thoughts in order to create a tailored plan to improve your overall mental well-being. We can work together to address the different challenges you are facing, and I am here to support you in your journey towards better mental health. your feelings or thoughts you may have regarding your mental health. Remember, mental health is a complex and unique experience for each individual, and our goal is to work collaboratively in identifying solutions that resonate with you., your statement about your color preferences may not necessarily be a direct reflection of your mental health, as preferences can be influenced by various factors. However, if these color choices are causing discomfort or negatively impacting your well-being, it would be important to explore the reasons behind your dislike for blue and what you can

/tmp/ipykernel_46178/3351683328.py:64: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  results = table.search(embed(message)).limit(2).to_df()



> Finished chain.

RESPONSE: Our preferences for colors can be subjective and may not necessarily indicate any underlying mental health issues. However, if it affects your well-being or causes discomfort in your daily life, it might be beneficial to explore the reasons behind your dislike for blue and what you can do to improve your feelings towards this color. Ultimately, addressing your feelings on color preferences should be secondary to dealing with your mental health concerns, which we can work on together to achieve better mental well-being.ments and complaints, please note that we have already covered some of these issues in our previous conversations. Nowments and complaints, please note that we have already covered some of these issues in our previous conversations. Nowments and complaints, please note that we have already covered some of these issues in our previous conversations. Nowments and complaints, please note that we have already covered some of these issues in our p

In [8]:
from pyngrok import ngrok

In [12]:
!killall ngrok

ngrok: no process found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
ngrok.set_auth_token("1s3kDMIreFHsV3FLZLNrm1Wi1py_4Xj8KfKQbuAUCnpMnPBF8")
# public_url = ngrok.connect(5000).public_url

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
app = Flask(__name__)
CORS(app, origins=["*"])
public_url = ngrok.connect(5000).public_url

@app.route('/', methods=['GET'])
def welcome():
  return "Welcome to Project <b>WeHeal</b>, developed as part of <b>Tree Hacks 2024</b>"

@app.route('/startSession/<user_id>', methods=['POST'])
def start_session(user_id):
  pdf_file = request.files['pdfFile']
  pdf_file.save('./user_details.pdf')
  user_text = load_document('./user_details.pdf')
  
  #user_text = request.json["text"]
  init_user_history(user_id)
  user_summary = get_user_summary(user_id, user_text)
  save_user_data(user_id, user_text)
  print("Summary: ", user_summary)
  return jsonify({"summary": user_summary})

@app.route('/api/ongoingSession/<user_id>', methods=['POST'])
def ongoing_session(user_id):
  user_reply = request.json["user_reply"]
  response = rag_answer(user_reply)
  return jsonify({"AIResponse": response})

@app.route('/api/saveConversation/<user_id>', methods=['POST'])
def save_conversation(user_id):
  user_reply = request.json["user_reply"]
  counselor_reply = request.json["counselor_reply"]
  save_conversation_to_history(user_id, user_reply, counselor_reply)
  return ""

if __name__ == '__main__':
  print("Please click " + public_url)
  app.run(port = 5000)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Please click https://5ea3-146-152-226-61.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2024 20:14:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2024 20:14:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Feb/2024 20:14:17] "GET / HTTP/1.1" 200 -


Started parsing the file under job_id 628dbfa9-cc79-4c71-adae-a05cb17f7447


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 
### Input:
Context: Assume you are a mental health professional with 10+ years experience. You will get chat based questions from a patient. Based on the question, you must generate responses for the user.
  Make absolutely sure all your responses seem like actual responses from a mental health expert.
  Do not suggest the patient to seek alternatives or to seek other counsellors or therapists or mental health
  The input and output formats are defined below. Only reply based on the output format (replace <reply> with your response).

Question: Based on the text below, summarize the pat

Created a chunk of size 1403, which is longer than the specified 700
Created a chunk of size 1667, which is longer than the specified 700
Created a chunk of size 2870, which is longer than the specified 700



> Finished chain.


127.0.0.1 - - [17/Feb/2024 20:15:08] "POST /startSession/user2222 HTTP/1.1" 200 -


data:  <class 'pyarrow.lib.Table'>
Table counsellor does not exists
Summary:  In this pre-counseling questionnaire, there are several central themes that Sanjay has discussed. He has a history of mental health issues, particularly anxiety, in his family. His job has also been a source of stress, contributing to his daily anxiety. He has undergone various treatments, including physical therapy and consultation with professionals. Sanjay's main concerns include managing his stress and anxiety, improving sleep quality, and finding better ways to cope with distress. He feels that working with a mental health expert like you could help him develop strategies and techniques to overcome these challenges and ultimately achieve a healthier work-life balance.
